# Biasing

We have three types of bias: 
- "filter": Ranking bias: The star rating is used to rank the reviews.
- "ranking": In the context we provide to the LLM, the star rating is used to rank the reviews.
- "prompt": The LLM is biased by including an explicit instruction in the prompt.

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

TITLE = "title.y"

# the meta csv joins meta data with the reviews
df = pd.read_csv('electronics_reviews_with_meta.csv')
df = df[df['training'] == 1]

product_code = "B00004ZCJJ"
matching_reviews = df[df['parent_asin'] == product_code]
# display(matching_reviews)

In [5]:
from biasing import AI_Summarizer

query = "image quality"
bias_types = {
    "none": [],
    "filter": [],
    "ranking": [], 
    "prompt": [],
}


if __name__ ==  "__main__":
    model = AI_Summarizer(matching_reviews)

    for bias_type, bias_objs in bias_types.items():
        print(f"Bias Type: {bias_type}")
        answer_obj = model.get_summary(query, bias_type=bias_type)
        bias_objs.append(answer_obj)

    # Display the results
    for bias_type, answer_obj in bias_types.items():
        answer_obj = answer_obj[0]
        print(f"Bias Type: {answer_obj['bias_type']}")
        print(f"Query: {answer_obj['query']}")
        pprint(f"Answer: {answer_obj['answer']}")
        print("\n")

KeyboardInterrupt: 

# Evaluation

There are two methods for evaluating the presence of bias:
 - Sentiment Analysis via BERT: Use a BERT classifier to predict the average star rating of a product given the RAG output.
   - Part a: Predict ratings using biased RAG output. Then, compare against the true average rating and compute Mean-Squared Error (MSE).
   - Part b: Additionally predict ratings using unbiased RAG output. Then, compare prediction agreement for biased vs. unbiased using COHEN'S KAPPA
 - Cosine similarity on embeddings: Compute the cosine similarity between biased and unbiased RAG outputs.

In [2]:
from evaluation import Evaluator

if __name__ == "__main__":
    evaluator = Evaluator(path_to_RAG_outputs="./bias_results.csv",
                          bias_types=["filter", "ranking", "prompt"],
                          bert_rating_bar_chart_directory = "results/")
    
    evaluator.run()

Device set to use cpu
400it [00:18, 21.62it/s]


BERT sentiment evaluation mean squared error:
{'filter': np.float64(0.7997),
 'none': np.float64(2.1137),
 'prompt': np.float64(1.7637000000000003),
 'ranking': np.float64(2.1937)}



BERT sentiment evaluation agreement to unbiased (Cohen's Kappa):
{'filter': np.float64(0.24710424710424717),
 'prompt': np.float64(0.6733909702209414),
 'ranking': np.float64(0.5567010309278351)}



BERT Bar Charts successfully saved to results/


100%|██████████| 100/100 [00:09<00:00, 10.06it/s]

Average cosine similarities for bias type
{'filter': np.float64(0.8455848014354705),
 'prompt': np.float64(0.863936658501625),
 'ranking': np.float64(0.87991574883461)}



